In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel
import requests
import json
import random
from collections import Counter
from keras.models import Sequential
from keras.layers import Dense
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [5]:
df = pd.read_csv("data/mpst_full_data.csv")
df = df[100:119]
df

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
100,tt0110632,Natural Born Killers,IntroductionMickey Knox and his wife Mallory g...,"comedy, cruelty, murder, neo noir, cult, viole...",test,imdb
101,tt1596350,This Means War,"CIA agents and best friends Franklin ""FDR"" Fos...","entertaining, revenge, murder, violence",train,wikipedia
102,tt1551632,Rizzoli & Isles,A young Hispanic woman prays to the Virgin Mar...,murder,test,imdb
103,tt0783598,Until Death,New Orleans narcotics detective Anthony Stowe ...,"neo noir, murder, cult, revenge, violence, mel...",val,wikipedia
104,tt0088757,Avenging Angel,"Set a few years after the first 'Angel' film, ...",cult,train,imdb
105,tt0414982,Final Destination 3,The movie opens with Wendy Christensen (Mary E...,"violence, horror, gothic, sadist",val,imdb
106,tt0102724,Psycho IV: The Beginning,"On a local radio show, the host and D.J. Fran ...","insanity, murder, flashback",train,imdb
107,tt3397884,Sicario,"In the opening shot, there is a text that expl...","suspenseful, neo noir, murder, violence, trage...",val,imdb
108,tt0901494,Life Is Hot in Cracktown,Based on writer/director Buddy Giovinazzo's no...,murder,val,imdb
109,tt0112288,The Addiction,New York University. Walking home after viewin...,"philosophical, cult, murder, violence",train,imdb


In [6]:
Api_key = 'k_dx4q6udk'
Api_key2 = 'k_b8vu8md1'
Api_key3 = 'k_g37c7xgx'

In [7]:
review_title = []
review_content = []
review_imdb_id = []
review_rating = []

for i in df['imdb_id']:
    url = 'https://imdb-api.com/en/API/Reviews/'+Api_key3+'/'+i

    response_API = requests.get(url)
    data = response_API.text
    for review in json.loads(data)['items']:
        review_imdb_id.append(i)
        review_rating.append(review['rate'])
        review_title.append(review['title'])
        review_content.append(review['content'])

movies_review = pd.DataFrame({'imdb_id': review_imdb_id, 'rate': review_rating ,'title': review_title, 'review': review_content})
movies_review


,imdb_id,rate,title,review
0,tt0110632,,good messages get lost in all the raucous,From director Oliver Stone comes this flashy b...
1,tt0110632,10,"Hypnotic, but...",...people really need to take another look at ...
2,tt0110632,4,Oliver Stone(d),Wow! Oliver Stone makes a David Lynch film...a...
3,tt0110632,7,A bizarre acid trip of a film that has good an...,Oliver Stone seems to have outdone himself on ...
4,tt0110632,7,You'll love it or hate it.,"I remember ""Natural Born Killers"" making a hug..."
...,...,...,...,...
437,tt0974661,8,Gotta Love A Zac Efron Movie,"This story certainly was different, Mike being..."
438,tt0974661,4,'17' never finds its own voice within Alternat...,17 Again is the latest entry in what I call th...
439,tt0974661,7,You don't know what you have until you lose it,"(Synopsis) In 1989, Mike O'Donnell (Zac Efron)..."
440,tt0974661,4,The land of confusion....,CHUGA CHUGA CHUGA - The Hollywood movie machin...


In [8]:
df_review_merge = df.merge(movies_review, on="imdb_id")
#Create csv to store new dataset
#df_review_merge.to_csv('data/movies_data.csv')
df_review_merge

,imdb_id,title_x,plot_synopsis,tags,split,synopsis_source,rate,title_y,review
0,tt0110632,Natural Born Killers,IntroductionMickey Knox and his wife Mallory g...,"comedy, cruelty, murder, neo noir, cult, viole...",test,imdb,,good messages get lost in all the raucous,From director Oliver Stone comes this flashy b...
1,tt0110632,Natural Born Killers,IntroductionMickey Knox and his wife Mallory g...,"comedy, cruelty, murder, neo noir, cult, viole...",test,imdb,10,"Hypnotic, but...",...people really need to take another look at ...
2,tt0110632,Natural Born Killers,IntroductionMickey Knox and his wife Mallory g...,"comedy, cruelty, murder, neo noir, cult, viole...",test,imdb,4,Oliver Stone(d),Wow! Oliver Stone makes a David Lynch film...a...
3,tt0110632,Natural Born Killers,IntroductionMickey Knox and his wife Mallory g...,"comedy, cruelty, murder, neo noir, cult, viole...",test,imdb,7,A bizarre acid trip of a film that has good an...,Oliver Stone seems to have outdone himself on ...
4,tt0110632,Natural Born Killers,IntroductionMickey Knox and his wife Mallory g...,"comedy, cruelty, murder, neo noir, cult, viole...",test,imdb,7,You'll love it or hate it.,"I remember ""Natural Born Killers"" making a hug..."
...,...,...,...,...,...,...,...,...,...
437,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,"romantic, humor",train,imdb,8,Gotta Love A Zac Efron Movie,"This story certainly was different, Mike being..."
438,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,"romantic, humor",train,imdb,4,'17' never finds its own voice within Alternat...,17 Again is the latest entry in what I call th...
439,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,"romantic, humor",train,imdb,7,You don't know what you have until you lose it,"(Synopsis) In 1989, Mike O'Donnell (Zac Efron)..."
440,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,"romantic, humor",train,imdb,4,The land of confusion....,CHUGA CHUGA CHUGA - The Hollywood movie machin...


In [23]:
df_review_merge_load = pd.read_csv("data/movies_data.csv")
df_review_merge_load

,imdb_id,title_x,plot_synopsis,rate,review
0,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,9.0,In 1950s Sweden a 12 year old boy Ingemar (Ant...
1,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,NaN,This odd little film relates the story of Inge...
2,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,10.0,I finally got to watch this film from beginnin...
3,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,8.0,"""Mit liv sum hund"" directed by talented direct..."
4,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,10.0,"My Life As A Dog, now this is a total delight...."
...,...,...,...,...,...
1684,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,8.0,"This story certainly was different, Mike being..."
1685,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,4.0,17 Again is the latest entry in what I call th...
1686,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,7.0,"(Synopsis) In 1989, Mike O'Donnell (Zac Efron)..."
1687,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,4.0,CHUGA CHUGA CHUGA - The Hollywood movie machin...


In [22]:
df_review_merge_load = df_review_merge_load.append(df_review_merge)

C:\Users\jerem\AppData\Local\Temp/ipykernel_11240/25965955.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_review_merge_load = df_review_merge_load.append(df_review_merge)


In [24]:
#need create merge to compile new values
df_review_merge_load.to_csv('data/movies_data.csv', index=False)
df_review_merge_load

,imdb_id,title_x,plot_synopsis,rate,review
0,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,9.0,In 1950s Sweden a 12 year old boy Ingemar (Ant...
1,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,NaN,This odd little film relates the story of Inge...
2,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,10.0,I finally got to watch this film from beginnin...
3,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,8.0,"""Mit liv sum hund"" directed by talented direct..."
4,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,10.0,"My Life As A Dog, now this is a total delight...."
...,...,...,...,...,...
1684,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,8.0,"This story certainly was different, Mike being..."
1685,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,4.0,17 Again is the latest entry in what I call th...
1686,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,7.0,"(Synopsis) In 1989, Mike O'Donnell (Zac Efron)..."
1687,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,4.0,CHUGA CHUGA CHUGA - The Hollywood movie machin...


In [26]:
df_review_merge_cleaned = df_review_merge_load.drop(
    columns=['tags', 'split', 'synopsis_source', 'title_y']
)
df_review_merge_cleaned.to_csv('data/movies_data.csv', index=False)

KeyError: "['tags', 'split', 'synopsis_source', 'title_y'] not found in axis"

In [27]:
df_review_merge_cleaned

,imdb_id,title_x,plot_synopsis,rate,review
0,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,9.0,In 1950s Sweden a 12 year old boy Ingemar (Ant...
2,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,10.0,I finally got to watch this film from beginnin...
3,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,8.0,"""Mit liv sum hund"" directed by talented direct..."
4,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,10.0,"My Life As A Dog, now this is a total delight...."
5,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,8.0,This Swedish film offers a different kind of s...
...,...,...,...,...,...
437,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,8,"This story certainly was different, Mike being..."
438,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,4,17 Again is the latest entry in what I call th...
439,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,7,"(Synopsis) In 1989, Mike O'Donnell (Zac Efron)..."
440,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,4,CHUGA CHUGA CHUGA - The Hollywood movie machin...


In [28]:
mask = df_review_merge_cleaned['rate'] != ''
mask2 = ~df_review_merge_cleaned['rate'].isna()
df_review_merge_cleaned = df_review_merge_cleaned[mask]
df_review_merge_cleaned = df_review_merge_cleaned[mask2]
df_review_merge_cleaned

,imdb_id,title_x,plot_synopsis,rate,review
0,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,9.0,In 1950s Sweden a 12 year old boy Ingemar (Ant...
2,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,10.0,I finally got to watch this film from beginnin...
3,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,8.0,"""Mit liv sum hund"" directed by talented direct..."
4,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,10.0,"My Life As A Dog, now this is a total delight...."
5,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,8.0,This Swedish film offers a different kind of s...
...,...,...,...,...,...
437,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,8,"This story certainly was different, Mike being..."
438,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,4,17 Again is the latest entry in what I call th...
439,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,7,"(Synopsis) In 1989, Mike O'Donnell (Zac Efron)..."
440,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,4,CHUGA CHUGA CHUGA - The Hollywood movie machin...


In [31]:
df_review_merge_cleaned['rate'] = df_review_merge_cleaned['rate'].astype(int)
df_review_merge_cleaned.dtypes

imdb_id          object
title_x          object
plot_synopsis    object
rate              int32
review           object
dtype: object

In [ ]:
#df_review_merge_cleaned['review'] = df_review_merge_cleaned['review'].astype('string')
#df_review_merge_cleaned.dtypes

In [32]:
df_review_merge_cleaned.rate

0       9
2      10
3       8
4      10
5       8
       ..
437     8
438     4
439     7
440     4
441     8
Name: rate, Length: 1471, dtype: int32

In [33]:
df_review_merge_cleaned.rate = df_review_merge_cleaned.rate/10

In [34]:
# A revoir autre method plus precise ?
df_review_merge_cleaned.rate[df_review_merge_cleaned.rate == 0.5] = 0
df_review_merge_cleaned.rate[df_review_merge_cleaned.rate == 0.4] = -0.25
df_review_merge_cleaned.rate[df_review_merge_cleaned.rate == 0.3] = -0.50
df_review_merge_cleaned.rate[df_review_merge_cleaned.rate == 0.2] = -0.75
df_review_merge_cleaned.rate[df_review_merge_cleaned.rate == 0.1] = -1.0

df_review_merge_cleaned.rate[df_review_merge_cleaned.rate == 0.6] = 0.25
df_review_merge_cleaned.rate[df_review_merge_cleaned.rate == 0.7] = 0.50
df_review_merge_cleaned.rate[df_review_merge_cleaned.rate == 0.8] = 0.75
df_review_merge_cleaned.rate[df_review_merge_cleaned.rate == 0.9] = 1.0
df_review_merge_cleaned.rate

C:\Users\jerem\AppData\Local\Temp/ipykernel_11240/1460424651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_merge_cleaned.rate[df_review_merge_cleaned.rate == 0.5] = 0
C:\Users\jerem\AppData\Local\Temp/ipykernel_11240/1460424651.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_merge_cleaned.rate[df_review_merge_cleaned.rate == 0.4] = -0.25
C:\Users\jerem\AppData\Local\Temp/ipykernel_11240/1460424651.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

0      1.00
2      1.00
3      0.75
4      1.00
5      0.75
       ... 
437    0.75
438   -0.25
439    0.50
440   -0.25
441    0.75
Name: rate, Length: 1471, dtype: float64

In [35]:

model = SentimentIntensityAnalyzer()

def get_sentiment(review):
    scores = model.polarity_scores(review)
    print(scores)
    return scores.get('compound')

df_review_merge_cleaned['vader'] = np.vectorize(get_sentiment)(df_review_merge_cleaned['review'])
df_review_merge_cleaned['vader_positive'] = (df_review_merge_cleaned.vader > 0).astype(bool)
df_review_merge_cleaned['real_positif'] = (df_review_merge_cleaned.rate > 0).astype(bool)


{'neg': 0.034, 'neu': 0.761, 'pos': 0.205, 'compound': 0.9943}
{'neg': 0.034, 'neu': 0.761, 'pos': 0.205, 'compound': 0.9943}
{'neg': 0.067, 'neu': 0.804, 'pos': 0.129, 'compound': 0.975}
{'neg': 0.011, 'neu': 0.726, 'pos': 0.263, 'compound': 0.9914}
{'neg': 0.068, 'neu': 0.84, 'pos': 0.092, 'compound': 0.8866}
{'neg': 0.071, 'neu': 0.745, 'pos': 0.184, 'compound': 0.9607}
{'neg': 0.107, 'neu': 0.744, 'pos': 0.148, 'compound': 0.8238}
{'neg': 0.105, 'neu': 0.811, 'pos': 0.085, 'compound': -0.5853}
{'neg': 0.068, 'neu': 0.736, 'pos': 0.196, 'compound': 0.9815}
{'neg': 0.115, 'neu': 0.749, 'pos': 0.136, 'compound': 0.8333}
{'neg': 0.024, 'neu': 0.738, 'pos': 0.238, 'compound': 0.9907}
{'neg': 0.035, 'neu': 0.813, 'pos': 0.152, 'compound': 0.9682}
{'neg': 0.056, 'neu': 0.733, 'pos': 0.211, 'compound': 0.9989}
{'neg': 0.028, 'neu': 0.804, 'pos': 0.168, 'compound': 0.9943}
{'neg': 0.068, 'neu': 0.83, 'pos': 0.102, 'compound': 0.7464}
{'neg': 0.107, 'neu': 0.75, 'pos': 0.143, 'compound': 0.9

In [36]:
df_review_merge_cleaned

,imdb_id,title_x,plot_synopsis,rate,review,vader,vader_positive,real_positif
0,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,1.00,In 1950s Sweden a 12 year old boy Ingemar (Ant...,0.9943,True,True
2,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,1.00,I finally got to watch this film from beginnin...,0.9750,True,True
3,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,0.75,"""Mit liv sum hund"" directed by talented direct...",0.9914,True,True
4,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,1.00,"My Life As A Dog, now this is a total delight....",0.8866,True,True
5,tt0089606,Mitt liv som hund,The action takes place in the years 1958-1959 ...,0.75,This Swedish film offers a different kind of s...,0.9607,True,True
...,...,...,...,...,...,...,...,...
437,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,0.75,"This story certainly was different, Mike being...",0.9753,True,True
438,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,-0.25,17 Again is the latest entry in what I call th...,0.9951,True,False
439,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,0.50,"(Synopsis) In 1989, Mike O'Donnell (Zac Efron)...",0.9955,True,True
440,tt0974661,17 Again,Popular high school senior Mike O'Donnell (Zac...,-0.25,CHUGA CHUGA CHUGA - The Hollywood movie machin...,0.9281,True,False


In [37]:
confusion_matrix = pd.crosstab(df_review_merge_cleaned['real_positif'], df_review_merge_cleaned['vader_positive'], rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)

Predicted  False  True
Actual                
False        154   130
True         233   954


In [38]:
from sklearn.metrics import classification_report
print(classification_report(df_review_merge_cleaned['real_positif'], df_review_merge_cleaned['vader_positive']))

              precision    recall  f1-score   support

       False       0.40      0.54      0.46       284
        True       0.88      0.80      0.84      1187

    accuracy                           0.75      1471
   macro avg       0.64      0.67      0.65      1471
weighted avg       0.79      0.75      0.77      1471



In [39]:
df_review = df_review_merge_cleaned.groupby(
    ['imdb_id', 'title_x', 'plot_synopsis'], as_index=False)['vader'].mean()

In [40]:
df_review

,imdb_id,title_x,plot_synopsis,vader
0,tt0021079,Little Caesar,Small-time Italian-American criminals Caesar E...,0.365932
1,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",0.896595
2,tt0035140,"Now, Voyager",Drab Charlotte Vale (Bette Davis) is an unattr...,0.862630
3,tt0036696,The Canterville Ghost,"In 1634, in England, Sir Simon de Canterville ...",0.470472
4,tt0036868,The Best Years of Our Lives,"After World War II, Fred Derry (Dana Andrews),...",0.940239
...,...,...,...,...
65,tt3397884,Sicario,"In the opening shot, there is a text that expl...",0.579426
66,tt3832914,War Room,Ms. Clara (Karen Abercrombie) is standing by a...,0.423996
67,tt4226388,Victoria,"Berlin, 04:00. After having her club time Vict...",0.318013
68,tt4731136,A Cure for Wellness,"In the opening scene, a man named Morris (Crai...",0.691935


In [41]:

tfv = TfidfVectorizer(
    min_df=3,
    max_features=None,
    strip_accents="unicode",
    analyzer="word",
    token_pattern="\w{1}",
    ngram_range=(1, 3),
    stop_words="english",
)

tfv_matrix = tfv.fit_transform(df_review["plot_synopsis"])
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)
indices = pd.Series(df_review.index, index=df_review['title_x']).drop_duplicates()

def give_rec(title, vader, sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    sig_scores = sig_scores[1:11]
    movie_indices = [i[0] for i in sig_scores]
    return df_review[['title_x','vader']].iloc[movie_indices][df_review["vader"] > vader]



print(give_rec('Dracula 2000', 0.7))

c:\Users\jerem\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn(


     title_x     vader
51  17 Again  0.917984


C:\Users\jerem\AppData\Local\Temp/ipykernel_11240/1432394100.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_review[['title_x','vader']].iloc[movie_indices][df_review["vader"] > vader]
